#### codes are fixed for InferenceModel, all its models, and inference each model for each dataset.

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00


In [2]:
import os
import json
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['KAGGLE_KEY'] = user_secrets.get_secret("KAGGLE_KEY")
os.environ['KAGGLE_USERNAME'] = user_secrets.get_secret("KAGGLE_USERNAME")


In [3]:
os.makedirs('/kaggle/dataset/', exist_ok=True)


# # Change below
# meta = dict(
#     id="nayechz/raw-gemma2-predict",
#     title="raw_gemma2_predict",
#     isPrivate=True,
#     licenses=[dict(name="other")]
# )

# with open('/kaggle/dataset/dataset-metadata.json', 'w') as f:
#     json.dump(meta, f)

In [4]:
# created_dir = "/kaggle/dataset/raw_gemma2_predict"
# os.makedirs(created_dir, exist_ok=True)
# import torch
# # tensor = torch.randn(1,2)
# tensor = torch.load("/kaggle/working/raw_gemma2_predict_massive200.pt")
# torch.save(tensor, f"/kaggle/dataset/raw_gemma2_predict/raw_gemma2_predict_massive200.pt")
# tensor = torch.load("/kaggle/working/raw_gemma2_predict_xcopa200.pt")
# torch.save(tensor, f"/kaggle/dataset/raw_gemma2_predict/raw_gemma2_predict_xcopa200.pt")
# !kaggle datasets create -p "/kaggle/dataset" --dir-mode zip
# # !kaggle datasets version -p "/kaggle/dataset" -m "Added new file raw_gemma2_predict_massive200.pt"


In [5]:
# !ls -lh /kaggle/dataset
# !rm -rf neurons neurons.pt raw_gemma2_predict raw_gemma2_predict_massive200 raw_gemma2_predict_massive200.pt

In [6]:
# !rm -rf /kaggle/dataset/raw_gemma2_predict_massive200 


In [7]:
# !mv /kaggle/working/raw_gemma2_predict_massive200.pt /kaggle/dataset/raw_gemma2_predict

In [8]:
# !kaggle datasets version -p /kaggle/working/raw_gemma2_predict_massive200 -m "Added new file raw_gemma2_predict_massive200.pt"
# 

In [9]:
# x = torch.load("/kaggle/input/raw-gemma2-predict/raw_gemma2_predict/raw_gemma2_predict_massive200.pt")
# x.shape

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
import evaluate
import pandas as pd
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class InferenceModel:
    def __init__(self, model_name, device):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", offload_buffers=True)
        self.acc_metric = evaluate.load("accuracy")  
        self.precision_metric = evaluate.load("precision")  
        self.recall_metric = evaluate.load("recall") 
        
        self.df = pd.DataFrame(columns=['pred_answer', 'pred_answertext'])
        self.predictions = []
        self.references = []
        self.answers = []
        self.eval_results = []
        self.model.eval()
        self.logits = []
    def init_eval(self):
        self.acc_metric = evaluate.load("accuracy")  
        self.precision_metric = evaluate.load("precision")  
        self.recall_metric = evaluate.load("recall") 
        self.df = pd.DataFrame(columns=['pred_answer', 'pred_answertext'])
        self.predictions = []
        self.references = []
        self.answers = []
        self.eval_results = []
        
    def get_logprobs(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model(**inputs, output_hidden_states=True)
        logits = outputs.logits
        # Shape remains, but values are now normalized as log probabilities over the vocabulary.
        logprobs = torch.gather(F.log_softmax(logits, dim=2), 2, output_ids.unsqueeze(2))
        return logprobs, outputs
        
    def inference(self, prompt, max_new_tokens=1):
        """
        Performs inference on a given prompt.
        Returns the decodede output
        """
        generated_text = ''
        len_sentence = 0
        if self.model_name == 'Qwen/Qwen2.5-0.5B-Instruct' or self.model_name == 'Qwen/Qwen2.5-3B-Instruct':
            # WARNING: messages ini harus sama dengan messages di check_index_prompt
            messages = [
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
            
            generated_ids = self.model.generate(
                    **model_inputs,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None,
                    max_new_tokens=max_new_tokens
                )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            len_sentence = len(self.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)["input_ids"])
        elif self.model_name == "google/gemma-2-2b-it":
            # WARNING: messages ini harus sama dengan messages di check_index_prompt
            messages = [
            {"role": "user", "content": prompt}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            model_inputs = self.tokenizer([text], return_tensors="pt", add_special_tokens=False).to(self.model.device)
            # print(f"(model_inputs['input_ids'] {(model_inputs['input_ids'])}")
            
            # print(f"len(model_inputs['input_ids'] {len(model_inputs['input_ids'])}")
            sentence_token_texts = self.tokenizer.convert_ids_to_tokens((model_inputs['input_ids'])[0])
            # print(f"sentence_token_texts {sentence_token_texts}")
            
            
            generated_ids = self.model.generate(
                    **model_inputs,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None,
                    max_new_tokens=max_new_tokens
                )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            len_sentence = len(self.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)["input_ids"])
            # print()
            
        else:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            len_sentence = len(inputs["input_ids"])
            output = self.model.generate(
                **inputs, 
                do_sample=False,
                temperature=None,
                top_p=None,
                top_k=None,
                max_new_tokens=1)
            # print(f"output: {output}")
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        
        self.answers.append(generated_text)
        # answer = "1" if "option1" in generated_text else "2" if "option2" in generated_text else None
        
        return generated_text, len_sentence

    def batch_inference(self, prompts):
        """
        Perform batch inference.
        """
        inputs = self.tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=50)
        generated_texts = []
        for input_ids, output_ids in zip(inputs.input_ids, outputs):
            generated_ids = [
                output_ids[len(input_ids):]
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            generated_texts.append(generated_text)
        self.answers.extend(generated_texts)
        return generated_texts

    def eval(self):
        """
        Evaluates the predictions against the references 
        """
        accuracy_results = self.acc_metric.compute(predictions=self.predictions, references=self.references)

        precision_results = self.precision_metric.compute(predictions=self.predictions, references=self.references, average="macro")

        recall_results = self.recall_metric.compute(predictions=self.predictions, references=self.references, average="macro")

        results = {
            'acc' : accuracy_results,
            'prec' : precision_results,
            'recall' : recall_results
        }
        self.eval_results.append(results)
        print("Evaluation Results:\n", results)
        return results

    
    def parse_save_output(self, generated_text, unique_labels, current_labeltexts, currentlabel):
        """
        generated_text: output from model.generate (e.g. '1', 'option 1', 'anna')
        unique_labels: unique labels from dataset (list of string e.g. ["1", "2"])
        current_labeltexts: the current label options (dict of current label text e.g. {1:'anna', 2:'susan'})
        currentlabel = the current reference label from dataset (string, the dataset['answer']. e.g. "2", "1" )
        """
        pred = None
        clean_generated_text = generated_text.lower().strip()
        for i in unique_labels:
            if str(i) in clean_generated_text or current_labeltexts[int(i)].lower().strip() in clean_generated_text or current_labeltexts[int(i)].lower().removeprefix("the").strip() in clean_generated_text :
                pred = i
                break
        # print(f"gt: {generated_text}")
        # print(f"pred : {pred}")
        if pred == None:
            pred = "9"
        self.predictions.append(pred)
        self.references.append(currentlabel)
        ans_dict = ({'pred_answer':pred, 'pred_answertext' : generated_text})
        # print(f"ans_dict {ans_dict}")
        self.df.loc[len(self.df)] = ans_dict
        return pred



In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from huggingface_hub import login

login("***REMOVED***")


In [13]:
from huggingface_hub import whoami

print(whoami()) 

{'type': 'user', 'id': '6759448ff3cd6492d40882fc', 'name': 'inayarhmns', 'fullname': 'Inaya Rahmanisa', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/Fb4wNhEjZbnQuce9_Lo3I.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'gemma', 'role': 'fineGrained', 'createdAt': '2025-02-20T09:42:36.221Z', 'fineGrained': {'canReadGatedRepos': True, 'global': [], 'scoped': [{'entity': {'_id': '6759448ff3cd6492d40882fc', 'type': 'user', 'name': 'inayarhmns'}, 'permissions': ['repo.content.read']}]}}}}


In [14]:
# massive_gemma = InferenceModel("google/gemma-2-2b-it", device)

In [15]:
# xcopa_gemma = InferenceModel("google/gemma-2-2b-it", device)

In [16]:
# massive_qwen = InferenceModel("Qwen/Qwen2.5-3B-Instruct", device)


In [17]:
xcopa_qwen = InferenceModel("Qwen/Qwen2.5-0.5B-Instruct", device)


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [18]:
# massive_qwen.model

In [19]:
# xcopa_gemma.model

The neurons we're interested is in act_fn in Qwen2MLP

In [20]:
# import torch 
# from datasets import get_dataset_config_names
# from tqdm import tqdm

# def inference_xwinograd(model_name, infer_model):
#     eval_result = {}
#     dataset_name = "AmazonScience/massive"
#     configs = get_dataset_config_names(dataset_name)
#     for lang in configs if not lang.startswith("all") :
#         ds = load_dataset(dataset_name, lang, split="test")
#         for data in tqdm(ds, desc=f"Processing {lang} Examples", leave=False):
#             # print(data)
#             current_labeltexts = {1:data['option1'], 2:data['option2']}
#             prompt = (
#                     f"{data['sentence']}\n"
#                     # f"Does _ refer to {current_labeltexts[2]}? True or False?"
#                     f"Who does '_' refer to? The answer should be one of '1: {current_labeltexts[1]}' or '2: {current_labeltexts[2]}'.\n"
#                     f"Answer:"
            
#                 )
#             # print(prompt)
#             generated_text = infer_model.inference(prompt)
#             # print(f"gen_text {generated_text}")
#             answer = generated_text.split()[0]
#             print(generated_text)
            
#             pred = infer_model.parse_save_output(
#                 generated_text, 
#                 ds['answer'], 
#                 current_labeltexts, 
#                 data['answer'])
#             break # one sample
        
#         evaluated = infer_model.eval()
#         eval_result[lang] = evaluated
#         print(f"Done with lang: {lang}")
#         break # one language
#     infer_model.df.to_csv(f"out_{model_name.split('/')[-1]}_{dataset_name.split('/')[-1]}", index=False)
#     return eval_result


# masukin hasil inference ke tabel

In [21]:
# infer_model = massive_qwen

- ada 2 dict untuk setiap language: 
-> *raw_values_avg_tokens* dan *raw_values_last_token*:

key-nya nama layer dari layer pertama sampe terakhir. setiap valuenya
ukurannya (num_row_in_dataset, hidden_dim) ini neuronnya bisa averaged tokens atau last token. output keluaran suatu neuron adlaah (batch_size, seq_len, num_neurons_di_suatu_layer) tapi seq_len menjadi 1 karena diambil averagenya (raw_values_avg_tokens) atau last token (raw_values_last_token) dari seq) sehingga (num_row_in_dataset, 1, hidden_dim) dijadiin (num_row_in_dataset, hidden_dim) aja.


- tensor *full_raw_values_avg_tokens* dan *full_raw_values_last_token*: adalah tensor dari raw_values_avg_tokens atau raw_values_last_token yang nyatuin semua neurons dari semua layer. ukurannya (batch_size, num_neurons_total_seluruh_layer)


- tensor *full_raw_values*: adalah tensor yang nyatuin full_raw_values_avg_tokens dan full_raw_values_last_token. ukurannya (2, batch_size, num_neurons_total_seluruh_layer). yg pertama full_raw_values_avg_tokens kedua full_raw_values_last_token.


In [22]:
raw_values_avg_tokens = {}
raw_values_last_token = {}
full_raw_values_avg_tokens = []
full_raw_values_last_token = []
full_raw_values = []


def save_raw_vals_to_dict(name, dictname, savees):
    if not name in dictname:
        dictname[name] = savees
    else:
        dictname[name] = torch.cat([dictname[name], savees], dim=0)
    
def clean_hooks(infer_model):
    for i in range(len(infer_model.model.model.layers)):
        mlp = infer_model.model.model.layers[i].mlp
        mlp.act_fn._forward_hooks.clear()

def get_activation_mlp(name, id_prompt_start, id_prompt_end, sequence_length): 
    """
        name (str): buat namain layer
        is_averaged_tokens (boolean): true if avergaed across tokens, else get the last token
    """
    def hook_fn(module, input, output):
        global raw_values_avg_tokens
        global raw_values_last_token

        # print(f"output shape {output.shape}")
        assert output.size(1) == sequence_length, f"Mismatch: output.size(1) = {output.size(1)}, sequence_length = {sequence_length}"
        # print(f"Layer: {name}, output Shape: {output.shape}")
        # print("Output", output)
        # output is always (batch size, dim_neuron_in_a_layer)
        # if is_averaged_tokens: # simpen average value across tokens
        # print(f"output shape {output.shape}")
        # print(f"avg biasa {output.detach().half().mean(dim=1).shape}")
        # average across token utterance only
        avg_output = output[:,id_prompt_start-1:id_prompt_end,:].detach().half().mean(dim=1).cpu() 
        # print(f"avg khusus {avg_output.shape}")
        
        save_raw_vals_to_dict(name, raw_values_avg_tokens, avg_output)
        # else:
        last_output = output.half()[:, -1,:].cpu() # simpen the last token
        save_raw_vals_to_dict(name, raw_values_last_token, last_output)
        # print(f"ini raw_values_avg_token {raw_values_avg_tokens}")
        # print(len(raw_values_last_token.values()))
    return hook_fn

def concat_neuron_layers(raw_values_avg_tokens, raw_values_last_token):
    full_raw_values_avg_tokens = list(raw_values_avg_tokens.values())[0]
    full_raw_values_last_token = list(raw_values_last_token.values())[0]
    
    # print(full_raw_values_avg_tokens.shape)
    # print(full_raw_values_last_token.shape)
    for i in list(raw_values_avg_tokens.values())[1:]:
        # print(i.shape)
        full_raw_values_avg_tokens = torch.cat([full_raw_values_avg_tokens, i], dim=-1).cpu()
    for i in list(raw_values_last_token.values())[1:]:
        full_raw_values_last_token = torch.cat([full_raw_values_last_token, i], dim=-1).cpu()
    # print(f"shape full_raw_values_last_token {full_raw_values_last_token.shape}")
    return full_raw_values_avg_tokens, full_raw_values_last_token


def merge_avg_last(full_raw_values_avg_tokens, full_raw_values_last_token):
    # print(f"shape full_raw_values_last_token {full_raw_values_last_token.shape}")
    full_raw_values = torch.stack((full_raw_values_avg_tokens, full_raw_values_last_token), dim=0).cpu()
    return full_raw_values


def register_hook(infer_model, handlers, id_prompt_start, id_prompt_end, sequence_length): 
    # remove_hooks(handlers)  # Remove any existing hooks before adding new ones
    clean_hooks(infer_model)
    num_layers = len(infer_model.model.model.layers)
    remove_hooks(handlers)
    for i in range (num_layers):
        mlp = infer_model.model.model.layers[i].mlp
        handlers.append(mlp.act_fn.register_forward_hook(get_activation_mlp(f"mlp{i}_act_fn", id_prompt_start, id_prompt_end, sequence_length)))
    # print(infer_model.model.model.layers[1].mlp.act_fn._forward_hooks)

    # for handler in handlers:
    #     handler.remove()
    # return handlers


def remove_hooks(handlers):
    for handler in handlers:
        handler.remove()
    handlers.clear()
    

avg across token for utterance only see all tokens

In [23]:
# import torch 
# from datasets import get_dataset_config_names
# from tqdm import tqdm


# infer_model = massive_qwen

# num_layers = len(infer_model.model.model.layers)

# dataset_name = "AmazonScience/massive"
# configs = get_dataset_config_names(dataset_name, trust_remote_code=True)
# n_instances = 0
# n_lang = 0
# language_dict = {}
# for lang in [lang for lang in configs if not lang.startswith("all")]:
#     # if n_lang >= 200:
#     #     break
#     language_dict[n_lang] = lang 
#     ds = load_dataset(dataset_name, lang, split="test")
#     intents = ds.features['intent'].names
    
#     scenarios = ds.features['scenario'].names
#     for data in tqdm(ds, desc=f"Processing {lang} Examples", leave=False):
#         # if n_instances >= 200:
#         #     break
#         label_intent = intents[data['intent']]
#         label_intent_no = -1
#         scenario = scenarios[data['scenario']]
        
#         intent_options = [intent[intent.find('_')+1:] for intent in intents if intent.startswith(scenario)]
        
#         current_labeltexts = dict()
#         for i, label in enumerate(intent_options):
#             if label_intent.endswith(label):
#                 label_intent_no = i
#             current_labeltexts[i] = label
#         prompt_before = (
#             f"""
#             Instruction: Classify the intent of the following utterance.
#             Utterance: 
#             """
#             )
#         prompt_after = (
#             f"""
#             Instruction: Classify the intent of the following utterance.  
#             Utterance: {data['utt']}.  
#             """
#             )
#         prompt_whole = (
#             f"""
#             Instruction: Classify the intent of the following utterance.  
#             Utterance: {data['utt']}.  
#             Options: {scenario} {', '.join(intent_options)}. 
#             Intent: 
#             """
#             )
        
#         def check_index_prompt(when, prompt):
#             messages = [
#                 {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#                 {"role": "user", "content": prompt}
#                 ]
#             text = infer_model.tokenizer.apply_chat_template(
#                 messages,
#                 tokenize=False,
#                 add_generation_prompt=True
#             )
#             sentence_tokens = infer_model.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
#             sentence_ids = sentence_tokens["input_ids"]
#             sentence_token_texts = infer_model.tokenizer.convert_ids_to_tokens(sentence_ids)[:-5]
#             print(f"{when} sentence_token_texts {sentence_token_texts}")
#             return len(sentence_token_texts)
#             # word_tokens_whole = infer_model.tokenizer(data['utt'].split()[0], return_offsets_mapping=True, add_special_tokens=False)["input_ids"]
#             # word_token_texts_whole = infer_model.tokenizer.convert_ids_to_tokens(word_tokens_whole)
#             # print(f"word_tokens_whole {word_token_texts_whole}")

#         # for i in range (3):
#         id_prompt_start = check_index_prompt("prompt_before", prompt_before)
#         id_prompt_end = check_index_prompt("prompt_after", prompt_after)
#         check_index_prompt("prompt_whole", prompt_whole)

#         messages = [
#                 {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#                 {"role": "user", "content": prompt_whole}
#                 ]
#         text = infer_model.tokenizer.apply_chat_template(
#             messages,
#             tokenize=False,
#             add_generation_prompt=True
#         )
#         sentence_tokens = infer_model.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
#         sentence_ids = sentence_tokens["input_ids"]
#         sentence_token_texts = infer_model.tokenizer.convert_ids_to_tokens(sentence_ids)
#         print(f"indexed {sentence_token_texts[id_prompt_start-1:id_prompt_end]}")
        
#         n_instances += 1
#         # break
 
#     break

## avg across token for the utterance only

In [24]:
def check_index_prompt(infer_model, when, prompt, model_name): # TODO: method ini pindahin ke class infer_model dan hapus printnya 
    """
    return len tokenized text when it is cut (excluding the chat template), len tokenized text 
    """
    if model_name == 'Qwen/Qwen2.5-0.5B-Instruct' or model_name == 'Qwen/Qwen2.5-3B-Instruct':
        messages = [
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt}]
        text = infer_model.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        sentence_tokens = infer_model.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
        sentence_ids = sentence_tokens["input_ids"]
        sentence_token_texts = infer_model.tokenizer.convert_ids_to_tokens(sentence_ids)[:-5]
        # print(f"{when} sentence_token_texts {sentence_token_texts}. len(sentence_token_texts) {len(sentence_token_texts)}")
        return len(sentence_token_texts), len(sentence_ids)
    elif model_name == 'google/gemma-2-2b-it':
        messages = [
            {"role": "user", "content": prompt}
            ]
        text = infer_model.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        sentence_tokens = infer_model.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
        sentence_ids = sentence_tokens["input_ids"]
        sentence_token_texts = infer_model.tokenizer.convert_ids_to_tokens(sentence_ids)[:-5]
        # print(f"{when} sentence_token_texts {sentence_token_texts}. len(sentence_token_texts) {len(sentence_token_texts)}")
        return len(sentence_token_texts), len(sentence_ids)


def get_index_start_end_prompt(infer_model, data, model_name, dataset_name): # TODO: method ini pindahin ke class infer_model dan hapus printnya 
    """
    return id start, id end, len sentence tokenize, prompt whole
    buat massive
    """
    prompt_before = prompt_after = prompt_whole = ""
    if dataset_name == "AmazonScience/massive":
        utterance, options = data
        prompt_before = (
            f"""
            Instruction: Classify the intent of the following utterance.
            Utterance: 
            """
            )
        prompt_after = (
            f"""
            Instruction: Classify the intent of the following utterance.  
            Utterance: {utterance}.  
            """
            )
        prompt_whole = (
            f"""
            Instruction: Classify the intent of the following utterance.  
            Utterance: {utterance}.  
            Options: {options}. 
            Intent: 
            """
            )
    elif dataset_name == "xcopa":
        premise, choice1, choice2, question_type = data
        prompt_before = (
            f"""
            Premise: 
            """)
        prompt_after = (
            f"""
            Premise: {premise}
            """)
        prompt_whole = (
            f"""
            Premise: {premise}
            I'm hesitating between the two options. Help me choose the {question_type}: 
            - {choice1} 
            - {choice2}
            """)
    if prompt_before == "":
        print("dataset invalid")
    
    id_prompt_start, _ = check_index_prompt(infer_model, "prompt_before", prompt_before, model_name)
    id_prompt_end, _ = check_index_prompt(infer_model,"prompt_after", prompt_after, model_name)
    _, len_sentence = check_index_prompt(infer_model, "prompt_whole", prompt_whole, model_name)
    return id_prompt_start, id_prompt_end, len_sentence, prompt_whole


def cleanup():
    raw_values_avg_tokens.clear()
    raw_values_last_token.clear()
    full_raw_values_avg_tokens.clear()
    full_raw_values_last_token.clear()
    full_raw_values.clear()

### inference massive

In [25]:
import torch 
from datasets import get_dataset_config_names
from tqdm import tqdm


def get_neurons_massive(infer_model, max_instances):
    eval_result = {}
    all_languages = []
    num_layers = len(infer_model.model.model.layers)
    handlers = []
    # print(infer_model.model.model.layers[1].mlp.act_fn._forward_hooks)
    
    dataset_name = "AmazonScience/massive"
    configs = get_dataset_config_names(dataset_name, trust_remote_code=True)
    n_instances = 0
    n_lang = 0
    language_dict = {}
    # register_hook(infer_model, handlers)
    
    for lang in [lang for lang in configs if not lang.startswith("all")]:
        # if n_lang >= 200:
        #     break
        language_dict[n_lang] = lang 
        cleanup()
    
        
        ds = load_dataset(dataset_name, lang, split="test")
        intents = ds.features['intent'].names
        
        scenarios = ds.features['scenario'].names
        for data in tqdm(ds, desc=f"Processing {lang} Examples", leave=False):
            if n_instances >= max_instances:
                break
            label_intent = intents[data['intent']]
            label_intent_no = -1
            scenario = scenarios[data['scenario']]
            
            intent_options = [intent[intent.find('_')+1:] for intent in intents if intent.startswith(scenario)]
            
            current_labeltexts = dict()
            for i, label in enumerate(intent_options):
                if label_intent.endswith(label):
                    label_intent_no = i
                current_labeltexts[i] = label
            utterance = data['utt']
            options = f"{scenario} {', '.join(intent_options)}"
            detail_data = utterance, options
            id_prompt_start, id_prompt_end, len_sentence, prompt_whole = get_index_start_end_prompt(infer_model, detail_data, infer_model.model_name, "AmazonScience/massive")
            clean_hooks(infer_model)
            register_hook(infer_model, handlers, id_prompt_start, id_prompt_end, len_sentence)
            generated_text, len_sentence_model = infer_model.inference(prompt_whole)
            assert len_sentence == len_sentence_model, f"Mismatch len sentence model {len_sentence_model} and prompt {len_sentence}"
            clean_hooks(infer_model)
            remove_hooks(handlers)
            n_instances += 1
            # break
            # print(f"🔵 After inference, raw_values_avg_tokens keys: {list(raw_values_avg_tokens.keys())}")
        # print(f"🔵 After inference lang.uage, raw_values_avg_tokens: {list(raw_values_avg_tokens.values())}")
        full_raw_values_avg_tokens, full_raw_values_last_token = concat_neuron_layers(raw_values_avg_tokens, raw_values_last_token)
        full_raw_values = merge_avg_last(full_raw_values_avg_tokens, full_raw_values_last_token)
        all_languages.append(full_raw_values)
        # print(full_raw_values.shape)
        n_lang += 1
        print(f"Done with lang: {lang}")
        for handler in handlers:
            handler.remove()
        # if n_lang >= 2: break # 
        n_instances = 0
        # break
        # print(infer_model.model.model.layers[1].mlp.act_fn._forward_hooks)
    cleanup()
    return all_languages



### inference xcopa

In [26]:
import torch 
from datasets import get_dataset_config_names
from tqdm import tqdm


def get_neurons_xcopa(infer_model, max_instances=None):
    
    eval_result = {}
    all_languages = []
    num_layers = len(infer_model.model.model.layers)
    handlers = []
    # print(infer_model.model.model.layers[1].mlp.act_fn._forward_hooks)
    
    dataset_name = "xcopa"
    configs = get_dataset_config_names(dataset_name, trust_remote_code=True)
    n_instances = 0
    n_lang = 0
    language_dict = {}
    for lang in [lang for lang in configs if not lang.startswith("translation")]:
        
        language_dict[n_lang] = lang 
        cleanup()
    
        
        ds = load_dataset(dataset_name, lang, split="test")
        for data in tqdm(ds, desc=f"Processing {lang} Examples", leave=False):
            # print(data)
            question_type = "cause" if data['question'] == "cause" else "effect"
            data_detail = data['premise'], data['choice1'], data['choice2'], question_type
            
            if max_instances and n_instances >= max_instances:
                break
            
            # intent_options = [intent[intent.find('_')+1:] for intent in intents if intent.startswith(scenario)]
            
            current_labeltexts = {0: data['choice1'], 1: data['choice2']}       
            id_prompt_start, id_prompt_end, len_sentence, prompt_whole = get_index_start_end_prompt(infer_model, data_detail, infer_model.model_name, dataset_name)
    
            messages = [
            {"role": "user", "content": prompt_whole}
                ]
            text = infer_model.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            sentence_tokens = infer_model.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
            sentence_ids = sentence_tokens["input_ids"]
            # print(f"sentence_id {len(sentence_ids)}")
            clean_hooks(infer_model)
            register_hook(infer_model, handlers, id_prompt_start, id_prompt_end, len_sentence)
            generated_text, len_sentence_model = infer_model.inference(prompt_whole)
            assert len_sentence == len_sentence_model, f"Mismatch len sentence model {len_sentence_model} and prompt {len_sentence}"
            clean_hooks(infer_model)
            remove_hooks(handlers)
            n_instances += 1
            # break
            # print(f"🔵 After inference, raw_values_avg_tokens keys: {list(raw_values_avg_tokens.keys())}")
        # print(f"🔵 After inference lang.uage, raw_values_avg_tokens: {list(raw_values_avg_tokens.values())}")
        full_raw_values_avg_tokens, full_raw_values_last_token = concat_neuron_layers(raw_values_avg_tokens, raw_values_last_token)
        full_raw_values = merge_avg_last(full_raw_values_avg_tokens, full_raw_values_last_token)
        all_languages.append(full_raw_values)
        # print(full_raw_values.shape)
        n_lang += 1
        print(f"Done with lang: {lang}")
        remove_hooks(handlers)
        # for handler in handlers:
        #     handler.remove()
        # if n_lang >= 2: break # 
        n_instances = 0
        # break
    #     # print(infer_model.model.model.layers[1].mlp.act_fn._forward_hooks)
    cleanup()
    return all_languages


## Retrieve neurons for a model and a dataset

In [27]:
import psutil

def check_cpu_memory():
    mem = psutil.virtual_memory()
    print(f"Total Memory: {mem.total / 1024**3:.2f} GB")
    print(f"Available Memory: {mem.available / 1024**3:.2f} GB")
    print(f"Used Memory: {mem.used / 1024**3:.2f} GB")
    print(f"Memory Usage: {mem.percent}%")
check_cpu_memory()
!nvidia-smi

Total Memory: 31.35 GB
Available Memory: 28.20 GB
Used Memory: 2.70 GB
Memory Usage: 10.0%
Mon Mar 24 10:01:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   28C    P0             30W /  250W |    2257MiB /  16384MiB |      0%      Default |
|                                         |      

In [28]:
import torch

def clear_cuda_cache():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clear_cuda_cache()

In [29]:
def save_full_neurons(filename, all_languages):
    full_languages_raw_values = torch.stack(all_languages, dim=0).cpu() 
    full_languages_raw_values.shape
    torch.save(full_languages_raw_values, filename)

In [30]:
# def make_dataset(dataset_id, dataset_title, data_dir):
#     """
#     dataset_id: username/datasetid
#     dataset_title: title
#     data_dir: dir in kaggle/dataset/
#     """
#     created_dir = f"/kaggle/dataset/{data_dir}"
#     os.makedirs(created_dir, exist_ok=True)
#     # Change below
#     meta = dict(
#         id=dataset_id,
#         title=dataset_title,
#         isPrivate=True,
#         licenses=[dict(name="other")]
#     )
    
#     with open(f'{created_dir}/dataset-metadata.json', 'w') as f:
#         json.dump(meta, f)
    
#     os.makedirs(created_dir, exist_ok=True)
#     # import torch
#     # tensor = torch.randn(1,2)
#     # tensor = torch.load("/kaggle/working/raw_gemma2_predict_massive200.pt")
#     # torch.save(tensor, f"{created_dir}/raw_gemma2_predict_massive200.pt")
#     # tensor = torch.load("/kaggle/working/raw_gemma2_predict_xcopa200.pt")
#     # torch.save(tensor, f"{created_dir}/raw_gemma2_predict/raw_gemma2_predict_xcopa200.pt")
#     # !kaggle datasets create -p created_dir --dir-mode zip
#     # !kaggle datasets version -p "/kaggle/dataset" -m "Added new file raw_gemma2_predict_massive200.pt"


#### 🐸XCOPA QWEN

In [31]:
check_cpu_memory()

Total Memory: 31.35 GB
Available Memory: 28.20 GB
Used Memory: 2.70 GB
Memory Usage: 10.0%


In [21]:
all_languages_xcopa_qwen = get_neurons_xcopa(xcopa_qwen, None)

README.md:   0%|          | 0.00/21.8k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/12.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/41.8k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/40.7k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Done with lang: zh


In [31]:
# # ini 500 row per language CUDA out of memory cuman muat 28
# full_languages_raw_values_xcopa_qwen = torch.stack(all_languages_xcopa_qwen, dim=0).cpu() 
# # full_languages_raw_values.shape
# save_full_neurons("raw_qwen05_predict_xcopa200.pt", full_languages_raw_values_xcopa_qwen)

#### 🐸XCOPA GEMMA

In [32]:
check_cpu_memory()

Total Memory: 31.35 GB
Available Memory: 23.89 GB
Used Memory: 7.00 GB
Memory Usage: 23.8%


In [38]:
# all_languages_xcopa_gemma = get_neurons_xcopa(xcopa_gemma, 200)

Done with lang: et


Done with lang: ht


Done with lang: id


Done with lang: it


Done with lang: qu


...

Done with lang: ta


Done with lang: th


Done with lang: tr


Done with lang: vi


Done with lang: zh


In [35]:
# full_languages_raw_values_xcopa_gemma = torch.stack(all_languages_xcopa_gemma, dim=0).cpu() 

# save_full_neurons("raw_gemma2_predict_xcopa200.pt", all_languages_xcopa_gemma)

NameError: name 'all_languages_xcopa_gemma' is not defined

#### 🐸MASSIVE QWEN 3B

In [36]:
# print("a")

a


In [ ]:
# all_languages_massive_qwen3 = get_neurons_massive(massive_qwen, 200)

Done with lang: af-ZA


Done with lang: am-ET


Done with lang: ar-SA


Done with lang: az-AZ


Done with lang: bn-BD


...

Done with lang: de-DE


Done with lang: el-GR


Done with lang: en-US


Done with lang: es-ES


Processing fa-IR Examples:   3%|▎         | 88/2974 [00:12<07:35,  6.33it/s]

#### 🐸MASSIVE QWEN 05

In [40]:
# all_languages_massive_qwen = get_neurons_massive(massive_qwen, 200)

af-ZA%2Ftrain%2F0000.parquet:   0%|          | 0.00/785k [00:00<?, ?B/s]

af-ZA%2Fvalidation%2F0000.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

af-ZA%2Ftest%2F0000.parquet:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11514 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2033 [00:00<?, ? examples/s]

...

zh-TW%2Ftest%2F0000.parquet:   0%|          | 0.00/197k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11514 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2033 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2974 [00:00<?, ? examples/s]

Done with lang: zh-TW


In [41]:
# full_languages_raw_values_massive_qwen = torch.stack(all_languages_massive_qwen, dim=0).cpu() 

# save_full_neurons("raw_qwen05_predict_massive200.pt", all_languages_massive_qwen)

#### 🐸MASSIVE GEMMA

In [17]:
# massive_gemma.model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=

In [18]:
# all_languages_massive_gemma = get_neurons_massive(massive_gemma, 200)

README.md:   0%|          | 0.00/34.4k [00:00<?, ?B/s]

massive.py:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

af-ZA%2Ftrain%2F0000.parquet:   0%|          | 0.00/785k [00:00<?, ?B/s]

af-ZA%2Fvalidation%2F0000.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

af-ZA%2Ftest%2F0000.parquet:   0%|          | 0.00/217k [00:00<?, ?B/s]

...

zh-TW%2Ftest%2F0000.parquet:   0%|          | 0.00/197k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11514 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2033 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2974 [00:00<?, ? examples/s]

Done with lang: zh-TW


In [ ]:
del raw_values_avg_tokens
del raw_values_last_token
del full_raw_values_avg_tokens
del full_raw_values_last_token
del full_raw_values 

In [ ]:
!nvidia-smi

In [ ]:
import torch

def all_languages_dict_to_tensor(all_languages_dict):
    batch_size = 10000  # Adjust based on your memory constraints
    chunks = [torch.stack(all_languages_dict[i:i+batch_size], dim=0).cpu() 
              for i in range(0, len(all_languages_dict), batch_size)]
    full_languages_raw_values = torch.cat(chunks, dim=0)  # Reassemble after stacking
    full_languages_raw_values.shape
    del chunks
    return full_languages_raw_values

In [ ]:
# full_languages_raw_values_massive_qwen3 = torch.cat(chunks, dim=0)  # Reassemble after stacking
# full_languages_raw_values_massive_qwen3.shape

In [ ]:
# full_languages_raw_values_massive_qwen3.shape

In [ ]:
# del chunks

In [ ]:
full_languages_raw_values = all_languages_dict_to_tensor(all_languages_xcopa_qwen)

unused

In [28]:
# make_dataset("nayechz/raw_qwen3_predict1", "raw_gemma2_predict", "raw_gemma2_predict")

Invalid folder: created_dir


In [31]:
# !cp /kaggle/working/raw_gemma2_predict_xcopa200.pt /kaggle/dataset/raw_gemma2_predict

save to a new dataset

In [ ]:
# dir_name = "neuron_xcopa"
# dataset_name = "neuron-xcopa"
os.makedirs('/kaggle/dataset/neuron_xcopa', exist_ok=True)

torch.save(full_languages_raw_values, "/kaggle/dataset/neuron_xcopa/raw_qwen05_xcopa_all.pt", _use_new_zipfile_serialization=True)	

In [20]:
# torch.save(all_languages_massive_qwen3, "raw_qwen3_predict_massive2.pt", _use_new_zipfile_serialization=True)	

In [ ]:
# torch.save(all_languages_massive_qwen3, "/kaggle/dataset/raw_qwen3_predictv1/raw_qwen3_predict_massive200.pt", _use_new_zipfile_serialization=True)	

In [ ]:
# !ls -lh /kaggle/dataset/raw_qwen3_predictv1


In [7]:
from datasets import get_dataset_config_names

def get_language_dict(dataset_name):
    dataset_name = dataset_name
    configs = get_dataset_config_names(dataset_name, trust_remote_code=True)
    n_instances = 0
    n_lang = 0
    language_dict = {}
    # register_hook(infer_model, handlers)
    
    for lang in enumerate([lang for lang in configs if not lang.startswith("all") and not lang.startswith("translation")]):
        # if n_lang >= 2:
        language_dict[lang[0]] = lang[1]
        # print(lang)
    
    return language_dict

# massive_lang_dict = get_language_dict("AmazonScience/massive")
xcopa_lang_dict = get_language_dict("xcopa")
torch.save(xcopa_lang_dict, "/kaggle/dataset/neuron_xcopa/xcopa_lang_dict.pt", _use_new_zipfile_serialization=True)	

README.md:   0%|          | 0.00/34.4k [00:00<?, ?B/s]

massive.py:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/21.8k [00:00<?, ?B/s]

In [ ]:
# Change below

# os.makedirs('/kaggle/dataset/neuron_xcopa', exist_ok=True)

meta = dict(
    id="nayechz/neuron-xcopa",
    title="neuron-xcopa",
    isPrivate=True,
    licenses=[dict(name="other")]
)
with open('/kaggle/dataset/neuron_xcopa/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)


In [ ]:
# !kaggle datasets create -p "/kaggle/dataset/neuron_xcopa" --dir-mode zip

!kaggle datasets version -p "/kaggle/dataset/neuron_xcopa" -m "Added new file"

In [43]:
# import torch
# print(torch.cuda.is_available())  # Should return True
# print(torch.cuda.current_device())  # Should return a device ID (e.g., 0)
# print(torch.cuda.get_device_name(0))  # Should return your GPU name


True
0
Tesla P100-PCIE-16GB
